# Goals
**[Script]** QC → trim → QC → align → sort/index → QC → quantify → report (agg. QC)
- Align reads from RNAseq samples to a reference in order to quantify genes and transcripts
- **[Benchmark](https://www.nature.com/articles/s41598-020-76881-x):** RSEM / HTSeq Union for counting and TMM for normalization are the most accurate
- **RNAseq Exp. Design (key point is to run pilot study 1st):** [Guide #1](https://www.melbournebioinformatics.org.au/tutorials/tutorials/rna_seq_exp_design/rna_seq_experimental_design/) // [Guide #2](https://www.lexogen.com/planning-for-success-a-strategic-design-guide-for-rna-seq-experiments-in-drug-discovery/)
    - **RNAseq sample prep:** Use RNA extraction kit to isolate RNA, remove contaminants such as genomic DNA, enrich mRNA (polyA: magnetic beads, rRNA depletion: probes), synthesize cDNA, fragment cDNA, create library for NGS sequencing (cDNA + ligated adapters), use genome assembly reference to align cDNA reads (or de novo assemble)

**[Overall]**
Identify and account for issues during RNAseq sample prep, such as:
- **Degraded RNA:** low RNA integrity number (RIN) and shorter reads → use rRNA depletion instead of polyA selection
    - **Good RNA RIN 6-8:** RNA samples typically display two major peaks representing the 18S and 28S ribosomal RNA
    - **Degraded RNA RIN <6:** As total RNA degrades, the 18S and 28S rRNA peaks slowly disappear and the degradation products emerge in the region between the 18S and small RNAs
- **rRNA contamination:** high mapping rates to rRNA, since it is much more abundant than mRNA
- **Batch effects:** different sequencers, batches, etc. → [randomize samples](https://scilifelab.github.io/courses/ngsintro/1910/slides/rnaseq/presentation.html#6)
- **Low RNA input:** non-equimolar RNA across samples
- **DNA contamination:** high [DNA] → remove with DNase
- **PCR amplification bias:** usually for reads with neutral GC content
- **More:** non-random RNA fragmentation, adapter-dimer formation, primer bias (random seqs are best)

**[To Do]**
1. Add rRNA removal with SortMeRNA + Ribosomal DB like silvaDB for 18S (small) and 28S (large) or Rfam/GenBank/RefSeq for 5S (canonical)
2. Add `nextflow cloud` [in GCP](https://www.nextflow.io/docs/latest/google.html) // [nextflow.config](https://www.nextflow.io/docs/latest/config.html) // [nf-core RNAseq](https://github.com/nf-core/rnaseq)
3. Add `ht-seq --mode=union` to compare with RSEM and STAR gene counts ([htseq1](https://htseq.readthedocs.io/en/latest/tutorials/exon_example.html#tutorial-htseq), [htseq2](https://htseq.readthedocs.io/en/latest/htseqcount.html))

# Packages

In [1]:
#########################
### Standard Library ####
#########################
import os
import sys
import warnings
import subprocess
from glob import glob

######################
### Data Handeling ###
######################
import pandas as pd
import numpy as np
import VinlandPy as vp

####################
### Session Info ###
####################
import session_info

## Options

In [ ]:
warnings.simplefilter(action="once", category=Warning)

pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 200

## Functions

# Parameters

## Inputs

In [ ]:
project_name = "GSE206529"
input_path = f"./inputs/{project_name}"
output_path = f"./outputs/{project_name}"

r1_suffix = "_1"  # _1.fastq.gz
r2_suffix = "_2"  # _2.fastq.gz

ref_path = "/mnt/disks/resources/data/references/outputs/Homo_sapiens/ensembl/111/"
star_index_path = os.path.join(ref_path, "star_index/")
rsem_index_path = os.path.join(ref_path, "rsem_index/Homo_sapiens.GRCh38.111")
bed12_path = os.path.join(ref_path, "Homo_sapiens.GRCh38.111.bed12")

print_cmds_only = True
run_gene_body_coverage = False
run_junction_annotation = False

n_threads = vp.set_threads()
n_threads

In [ ]:
# Read in metadata with fastq paths
df_meta = vp.load_file("GSE206529_sample_sheet_for_alignment.csv")

read_ends = df_meta["library_layout"].str.lower().unique()[0]
avg_read_lengths = df_meta[["r1_read_length","r2_read_length"]].stack().mean()
sd_read_lengths = np.std(df_meta[["r1_read_length","r2_read_length"]].stack().tolist())
strandedness = "reverse"  # none, forward, reverse

read_ends

In [ ]:
# Raw reads
raw_fq_path = vp.create_dir(os.path.join(input_path,"raw_reads"))

if read_ends=="single":
    raw_fq_names2files = {os.path.basename(f).split(".")[0]: [f] for f in df_meta["r1_path"].tolist()}

if read_ends=="paired":
    raw_fq_names2files = {}
    for r1_path, r2_path in zip(df_meta["r1_path"], df_meta["r2_path"]):
        sample_id = os.path.basename(r1_path).split(".")[0].replace(r1_suffix,"")
        raw_fq_names2files[sample_id] = [r1_path, r2_path]

len(raw_fq_names2files)

In [ ]:
# Clean reads
clean_fq_path = vp.create_dir(os.path.join(input_path,"clean_reads"))

clean_fq_names2files = {sample_id: [path.replace("raw_reads/", "clean_reads/trimmed_") for path in paths]\
                        for sample_id, paths in raw_fq_names2files.items()}

len(clean_fq_names2files)

## Outputs

In [ ]:
qc_path = vp.create_dir(os.path.join(output_path,"qc"))
star_output_path = vp.create_dir(os.path.join(output_path,"star_output"))
rsem_output_path = vp.create_dir(os.path.join(output_path,"rsem_output"))

# QC: Raw Reads
- **FastQC** to assess raw reads for base quality, GC content, adapter contamination, duplication rates, and more
- Need more? `seqkit stats --all` can find min_len, avg_len, max_len, num_seqs, N50, AvgQual, and more

In [ ]:
raw_fq_files = [path for sample_id, paths in raw_fq_names2files.items() for path in paths]

cmd_fqc1 = (
    f"/mnt/disks/resources/software/FastQC/fastqc "
    f"--outdir {qc_path} "
    f"--threads {n_threads} "
    # f"--extract "  # Uncompress zipped output
    # f"--dir /mnt/disks/tmp/ "  # Path to store temp files; Default: System temp dir
    f"{" ".join(raw_fq_files)}"
)
if print_cmds_only:
    print(cmd_fqc1)
else:
    res_fqc1 = vp.run_cmd(cmd_fqc1)

# Trim Reads
- **[Fastp](https://github.com/OpenGene/fastp?tab=readme-ov-file#all-options)** (written in C++) to trim reads
    - **Remove adapter sequences (auto-detected)**
    - **Cut low quality bases** per read by evaluating mean quality from a sliding window (like Trimmomatic but faster)
    - **Filter out short reads** after trimming (ex. one-third of original length)
    - Correct mismatched base pairs in overlapped regions of paired end reads, if one base is with high quality while the other is with ultra low quality
    - *Optional: Trim poly-A/T tails. Remove reads with low complexity (too many Ns).*

In [ ]:
# Bash command: for fqpath in `ls SRX*/*`;  do fqdir=$(echo $fqpath | cut -d '/' -f1); fqname=$(echo $fqpath | cut -d '/' -f2 | cut -d'.' -f1); fastp -i $fqdir/$fqname.fastq.gz -o $fqdir/$fqname.clean.fastq.gz -w 3 --json fastp/$fqname.json --html fastp/$fqname.html --failed_out fastp/$fqname.failed.reads.txt; done
for fastq_name, fastq_file in raw_fq_names2files.items():    
    cmd_fastp = (
        f"/mnt/disks/resources/software/miniconda3/envs/Py-3.12/bin/fastp "
        f"--in1 {fastq_file[0]} "
        f"--out1 {clean_fq_names2files[fastq_name][0]} "
        f"--thread {n_threads} "
        f"--length_required {int(avg_read_lengths/3)} "
        f"--json {os.path.join(qc_path, f"{fastq_name}_fastp")}.json "
        f"--html {os.path.join(qc_path, f"{fastq_name}_fastp")}.html"
        # f"--stdout"  # Output passing-filters reads to STDOUT
        # f"--failed_out {}.failed.reads.txt "
        # f"–adapter_fasta "  # Path to the adapter FASTA file with adapter sequences to trim
        # f"--merge "  # For PE data, indicate whether to merge reads into a single read if they are overlapped
        # f"--merged_out {path} "  # For PE data, path to to store merged reads (note: not typically done) 
        # f"--length_limit {int(avg_read_lengths*1.5)}"  # Reads longer than length_limit will be discarded, default 0 means no limitation. (int [=0])
        # f"–trim_front1 N "  # Trim a fixed number of bases off the left end of read1
        # f"–trim_front2 N "  # Trim a fixed number of bases off the left end of read2
        # f"--trim_poly_x "  # Enable polyX trimming in 3' ends
    )
    
    if read_ends=="paired":
        cmd_fastp = (
            f"{cmd_fastp} "
            f"--in2 {fastq_file[1]} "
            f"--out2 {clean_fq_names2files[fastq_name][1]} "
            f"--detect_adapter_for_pe "  # Default: adapter sequence auto-detection is enabled for SE data only
        )
    if print_cmds_only:
        print(cmd_fastp)
    else:
        res_fastp = vp.run_cmd(cmd_fastp)

# QC: Post-Trimming
- **FastQC** to verify trimming effectiveness, such as improved per-base quality scores, reduced adapter content, and possible removal of overrepresented sequences (ex. poly-G tails in NovaSeq data)

In [ ]:
clean_fq_files = [path for sample_id, paths in clean_fq_names2files.items() for path in paths]

cmd_fqc2 = (
    f"/mnt/disks/resources/software/FastQC/fastqc "
    f"--outdir {qc_path} "
    f"--threads {n_threads} "
    f"{" ".join(clean_fq_files)}"
)
if print_cmds_only:
    print(cmd_fqc2)
else:
    res_fqc2 = vp.run_cmd(cmd_fqc2)

# Align Reads
**[STAR](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf)** to align reads to a reference genome
- STAR is an RNA-Seq aligner that stands for Spliced Transcripts Alignment to a Reference
    - It is optimized for gapped alignment across exon junctions and is considered the gold standard
- Since some reads will map to 2 different exons, they need to be split into two seeds
    - Sometimes when the second seed is extended there will be mismatches (ex. poly-A tail or adapter content), which may be soft clipped
- **Note:** STAR defines junction start/end as intronic bases, other software may define them as exonic bases
- **More:** STARsolo, STARconsensus, STARdiploid

In [ ]:
for fastq_name, fastq_file in clean_fq_names2files.items():    
    cmd_star = (
        f"/mnt/disks/resources/software/STAR-2.7.11b/source/STAR "
        f"--runMode alignReads "
        f"--runThreadN {n_threads} "
        f"--readFilesCommand zcat "
        f"--genomeDir {star_index_path} "
        f"--limitBAMsortRAM 20000000000 "
        f"--outFileNamePrefix {os.path.join(star_output_path,f"{fastq_name}_")} "
        f"--outSAMtype BAM SortedByCoordinate "  # If problematic, reduce --outBAMsortingThreadN from default 6 to lower values
        f"--outSJtype Standard "  # Set to None to disable writing of SJ.out.tab files
        f"--quantMode TranscriptomeSAM"  # Aligned.toTranscriptome.out.bam; GeneCounts (not used): coincide with those from htseq-count with default parameters
        # f"--clipAdapterType None "
        # f"--sjdbOverhang {int(max(read_lengths)-1)} "  # Error: --sjdbOverhang=60 is not equal to the value at the genome generation step =100
        # f"--outTmpDir /mnt/disks/resources "  # Error; Remove from boot disk, which only has 100 Gb
        # f"--peOverlapNbasesMin "  # More accurate mapping accuracy for paired-end libraries with short insert sizes
        # f"--twopassMode Basic "  # Need novel junctions? Idea is to run 1st pass of STAR mapping with the usual parameters, then collect the junctions detected in the first pass, and use them as ”annotated” junctions for the 2nd pass mapping"
        # f"--varVCFfile {vcf_path}"  # Path to custom VCF file
        # See "All options" section in manual for more. STAR has a crazy number of options.
    )

    if read_ends=="single":
        cmd_star = (
            f"{cmd_star} "
            f"--readFilesIn {fastq_file[0]}"
        )

    if read_ends=="paired":
        cmd_star = (
            f"{cmd_star} "
            f"--readFilesIn {fastq_file[0]} {fastq_file[1]}"
        )

    if print_cmds_only:
        print(cmd_star)
    else:
        res_star = vp.run_cmd(cmd_star)

# Index BAMs

In [ ]:
bam_files = glob(os.path.join(star_output_path, "**", "*"), recursive=True)
bam_files = [f for f in bam_files if f.endswith("Aligned.sortedByCoord.out.bam")]

bam_names2files = {os.path.basename(f).split("_")[0]: f for f in bam_files}
len(bam_names2files)

In [ ]:
for bam_file in bam_files:    
    cmd_bai = (
        f"/mnt/disks/resources/software/samtools-1.20/samtools index "
        f"--bai "
        f"--output {bam_file+".bai"} "
        f"--threads {n_threads} "
        f"{bam_file}"
    )
    if print_cmds_only:
        print(cmd_bai)
    else:
        res_bai = vp.run_cmd(cmd_bai)

# QC: Post-Alignment
**RSeQC** to assess strandedness, gene features, and RNA degradation (3' bias)
- **infer-experiment:** Unstranded means reads can map to either strand (for large scale expression; less expensive)
    - Strandedness preserves original strand info
        - For novel transcript discovery, overlapping genes, correct read assignment for alternative splice sites
- **genebody-coverage:** RNA degradation occurs from 5' to 3', so if there is a 3' bias (e.g. higher % of 3'-UTR in RSeqQC) it is an indication of poor RNA prep. This can lead to poor coverage, poorly mapped reads, and high MT content.
- **Note: Do not remove duplicate reads**, since these are biological duplicates (highly expressed genes), and removal risks losing valid data

In [ ]:
if run_gene_body_coverage:
    for bam_name, bam_file in bam_names2files.items():    
        cmd_rseqc1 = (
            f"/mnt/disks/resources/software/miniconda3/envs/Py-3.12/bin/geneBody_coverage.py "
            f"--refgene={bed12_path} "
            f"--input={bam_file} "
            f"--out-prefix={os.path.join(qc_path,f"{bam_name}")} "
            f"--format=png"
        )
        if print_cmds_only:
            print(cmd_rseqc1)
        else:
            res_rseqc1 = vp.run_cmd(cmd_rseqc1)

In [ ]:
for bam_name, bam_file in bam_names2files.items():    
    cmd_rseqc2 = (
        f"/mnt/disks/resources/software/miniconda3/envs/Py-3.12/bin/infer_experiment.py "
        f"--refgene={bed12_path} "
        f"--input={bam_file} "
        f"--sample-size=2000000 "
        f"> {os.path.join(qc_path, f"{bam_name}.inferExperiment.txt")}"
    )
    if print_cmds_only:
        print(cmd_rseqc2)
    else:
        res_rseqc2 = vp.run_cmd(cmd_rseqc2)

In [ ]:
for bam_name, bam_file in bam_names2files.items():    
    cmd_rseqc3 = (
        f"/mnt/disks/resources/software/miniconda3/envs/Py-3.12/bin/read_distribution.py "
        f"--refgene={bed12_path} "
        f"--input={bam_file} "
        f"> {os.path.join(qc_path, f"{bam_name}.readDistribution.txt")}"
    )
    if print_cmds_only:
        print(cmd_rseqc3)
    else:
        res_rseqc3 = vp.run_cmd(cmd_rseqc3)

In [ ]:
if run_junction_annotation:
    for bam_name, bam_file in bam_names2files.items():    
        cmd_rseqc4 = (
            f"/mnt/disks/resources/software/miniconda3/envs/Py-3.12/bin/junction_annotation.py "
            f"--refgene={bed12_path} "
            f"--input={bam_file} "
            f"--out-prefix={os.path.join(qc_path,f"{bam_name}")}"
        )
        if print_cmds_only:
            print(cmd_rseqc4)
        else:
            res_rseqc4 = vp.run_cmd(cmd_rseqc4)

# Quantify Genes & Transcripts
**[RSEM](https://github.com/deweylab/RSEM)** to quantify the expression of genes and transcript
- Handles reads that map to multiple genes or isoforms
- Useful for de novo transcriptome assemblies as it does not rely on a reference genome
- For paired-end data, RSEM will automatically learn a fragment length distribution from the data
- Need bigWig? `rsem-bam2wig sorted_bam_input wig_output wiggle_name`

**How does RSEM count a read that maps to more than one transcript of the same gene?**
- At the transcript level, RSEM assigns a fractional count of the read to each transcript, based on the expression estimates of those transcripts (expectation-maximization algorithm)
- At the gene level, RSEM sums the expected counts from all transcripts belonging to the same gene
- More accurate than discarding ambiguous reads or counting them multiple times

In [ ]:
bam_tx_files = glob(os.path.join(star_output_path, "**", "*"), recursive=True)
bam_tx_files = [f for f in bam_tx_files if f.endswith("Aligned.toTranscriptome.out.bam")]

bam_tx_names2files = {os.path.basename(f).split("_")[0]: f for f in bam_tx_files}
len(bam_tx_names2files)

In [ ]:
# Syntax: rsem-calculate-expression [options] --alignments [--paired-end] input reference_name sample_name
for bam_name, bam_file in bam_tx_names2files.items():    
    cmd_rsem = (
        f"/mnt/disks/resources/software/RSEM-1.3.3/rsem-calculate-expression "
        f"--alignments "
        f"--strandedness {strandedness} "
        f"--num-threads {n_threads} "
        f"--no-bam-output "  # --output-genome-bam: Only if BAM is genome-aligned
        # f"--chipseq-peak-file "  # See docs for more chipseq options
    )
    
    if read_ends=="single":
        cmd_rsem = (
            f"{cmd_rsem}"
            f"--fragment-length-mean {avg_read_lengths} "
            f"--fragment-length-sd {sd_read_lengths}"
        )
    
    if read_ends=="paired":
        cmd_rsem = (
            f"{cmd_rsem}"
            f"--paired-end"
        )
    
    cmd_rsem = (
        f"{cmd_rsem} "
        f"{bam_file} "
        f"{rsem_index_path} "
        f"{os.path.join(rsem_output_path, f"{bam_name}")}"
    )
    if print_cmds_only:
        print(cmd_rsem)
    else:
        res_rsem = vp.run_cmd(cmd_rsem)

# QC: Aggregate QC metrics
**[MultiQC](https://docs.seqera.io/multiqc/modules/)** to report all QC metrics across samples (uniquely mapped reads, duplicate reads, etc.)
- MultiQC is a reporting tool that parses and summarises statistics from bioinformatics tool outputs, such as log files and console outputs. 

In [ ]:
cmd_mqc = (
    f"/mnt/disks/resources/software/miniconda3/envs/Py-3.12/bin/multiqc "
    f"--filename multiqc_report_{project_name} "
    f"--outdir {qc_path} "
    f"{qc_path} {star_output_path}"  # Looks recursively through subdirs
    # f"--ai-summary "  # Requires token from seqera or openai
    # f"--export "
    # f"--title {report_tile}"
)
if print_cmds_only:
    print(cmd_mqc)
else:
    res_mqc = vp.run_cmd(cmd_mqc)

# Session info

In [2]:
session_info.show(os=True, std_lib=False, dependencies=False)

In [3]:
print(("\n").join(["-----", "FastQC    0.12.1", "fastp     0.23.2", "STAR      2.7.11b", "samtools  1.20", "RSeQC     5.0.4", "RSEM      1.3.3", "MultiQC   1.27.1", "-----"]))

-----
FastQC    0.12.1
fastp     0.23.2
STAR      2.7.11b
samtools  1.20
RSeQC     5.0.4
RSEM      1.3.3
MultiQC   1.27.1
-----
